Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0


# Notebook for Financial Fraud data exploration 
***Please download the [banksim data](https://www.kaggle.com/datasets/ealaxi/banksim1) from Kaggle*** for financial fraud use case following the instructions in Readme (or *notebooks/download_data.ipynb*) first in order to run all the notebooks related to the financial fraud use case

The BankSim dataset is a simulated 6-month dataset with ~587K clean transactions and 7200 fraud transactions. The first CSV is the raw transaction data, and the second CSV is the transactions organized as a graph which the customer and merchants being the nodes, and the transaction as the edge.

## Table of Contents
1. Load raw transaction data
  * Make observations on (customer, merchant) transactions
2. Load raw graph network data
  * Compare with raw data 

In [1]:
import pandas as pd
import numpy as np

## Load in raw transaction data

In [2]:
raw_data_path = '../../data/01_raw/financial_fraud/bs140513_032310.csv'

raw_trans_data = pd.read_csv(raw_data_path)

raw_trans_data.shape

(594643, 10)

In [3]:
print(raw_trans_data.columns)

Index(['step', 'customer', 'age', 'gender', 'zipcodeOri', 'merchant',
       'zipMerchant', 'category', 'amount', 'fraud'],
      dtype='object')


### Observation: raw trans data has more categorical variables(age, gender. zipcode) for customer and merchant(zip) than the network data

In [4]:
raw_trans_data.describe()

,step,amount,fraud
count,594643.000000,594643.000000,594643.000000
mean,94.986827,37.890135,0.012108
std,51.053632,111.402831,0.109369
min,0.000000,0.000000,0.000000
25%,52.000000,13.740000,0.000000
50%,97.000000,26.900000,0.000000
75%,139.000000,42.540000,0.000000
max,179.000000,8329.960000,1.000000


In [5]:
raw_trans_data_sorted = raw_trans_data.sort_values(by=['customer', 'step']).reset_index(drop=True)

In [6]:
raw_trans_data_sorted.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,30,'C1000148617','5','M','28007','M1888755466','28007','es_otherservices',143.87,0
1,38,'C1000148617','5','M','28007','M1741626453','28007','es_sportsandtoys',16.69,0
2,42,'C1000148617','5','M','28007','M1888755466','28007','es_otherservices',56.18,0
3,43,'C1000148617','5','M','28007','M840466850','28007','es_tech',14.74,0
4,44,'C1000148617','5','M','28007','M1823072687','28007','es_transportation',47.42,0


## Dive deeper into the transactions

### Observation: one customer can make multiple transactions at one merchant 

In [7]:
raw_trans_data_sorted.loc[
    (raw_trans_data_sorted.customer=="'C1093826151'")&(raw_trans_data_sorted.merchant=="'M348934600'")
].head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
31014,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
31016,2,'C1093826151','4','M','28007','M348934600','28007','es_transportation',37.21,0
31017,3,'C1093826151','4','M','28007','M348934600','28007','es_transportation',31.63,0
31018,4,'C1093826151','4','M','28007','M348934600','28007','es_transportation',35.86,0
31020,6,'C1093826151','4','M','28007','M348934600','28007','es_transportation',39.58,0


In [9]:
known_fraud = raw_trans_data_sorted.loc[raw_trans_data_sorted.fraud==1]
known_fraud.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
60,102,'C1000148617','5','M','28007','M480139044','28007','es_health',323.64,1
139,39,'C100045114','4','M','28007','M732195782','28007','es_travel',3902.93,1
145,87,'C100045114','4','M','28007','M2122776122','28007','es_home',52.11,1
146,87,'C100045114','4','M','28007','M1873032707','28007','es_hotelservices',39.86,1
188,137,'C100045114','4','M','28007','M1353266412','28007','es_hotelservices',960.66,1


In [10]:
known_fraud.shape

(7200, 10)

### Observation: same (customer, merchant) pair can be flagged as fraud multiple times 

In [11]:
known_fraud[known_fraud.duplicated(subset=['customer', 'merchant'])].head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
351,122,'C1001065306','1','M','28007','M480139044','28007','es_health',1024.36,1
361,144,'C1001065306','1','M','28007','M480139044','28007','es_health',18.18,1
362,153,'C1001065306','1','M','28007','M980657600','28007','es_sportsandtoys',270.77,1
363,155,'C1001065306','1','M','28007','M17379832','28007','es_sportsandtoys',230.36,1
1719,122,'C1007572087','2','F','28007','M732195782','28007','es_travel',7635.41,1


In [12]:
known_fraud.loc[(known_fraud.customer=="'C1001065306'")&(known_fraud.merchant=="'M980657600'")]

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
360,142,'C1001065306','1','M','28007','M980657600','28007','es_sportsandtoys',100.89,1
362,153,'C1001065306','1','M','28007','M980657600','28007','es_sportsandtoys',270.77,1


### Observation: for same customer on same category purchase, the fraud flag can be different 

In [13]:
raw_trans_data_sorted.loc[(raw_trans_data_sorted.customer=="'C1000148617'")&(raw_trans_data_sorted.category=="'es_health'")]

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
60,102,'C1000148617','5','M','28007','M480139044','28007','es_health',323.64,1
124,173,'C1000148617','5','M','28007','M1053599405','28007','es_health',165.23,0


## Load in the raw network data 

In [14]:
raw_net_data_path = '../../data/01_raw/financial_fraud/bsNET140513_032310.csv'

In [15]:
raw_net_data = pd.read_csv(raw_net_data_path)

In [16]:
raw_net_data.shape

(594643, 5)

In [17]:
raw_net_data.columns

Index(['Source', 'Target', 'Weight', 'typeTrans', 'fraud'], dtype='object')

### Observation: Source is the customer id, Target is the merchant id and Weight is the transaction amount

Most of the features are available in in the raw transaction data instead

In [18]:
raw_net_data.loc[(raw_net_data.Source=="'C1093826151'")&(raw_net_data.Target=="'M348934600'")].head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
5076,'C1093826151','M348934600',37.21,'es_transportation',0
8664,'C1093826151','M348934600',31.63,'es_transportation',0
9936,'C1093826151','M348934600',35.86,'es_transportation',0
15042,'C1093826151','M348934600',39.58,'es_transportation',0


In [19]:
raw_trans_data_sorted.loc[
    (raw_trans_data_sorted.customer=="'C1093826151'")
    &(raw_trans_data_sorted.merchant=="'M348934600'")
].head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
31014,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
31016,2,'C1093826151','4','M','28007','M348934600','28007','es_transportation',37.21,0
31017,3,'C1093826151','4','M','28007','M348934600','28007','es_transportation',31.63,0
31018,4,'C1093826151','4','M','28007','M348934600','28007','es_transportation',35.86,0
31020,6,'C1093826151','4','M','28007','M348934600','28007','es_transportation',39.58,0


# References

Edgar Alonso Lopez-Rojas and Stefan Axelsson. 2014. BANKSIM: A BANK PAYMENTS SIMULATOR FOR FRAUD DETECTION RESEARCH.